In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [3]:
max_features = 1000000
Vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode='tf_idf', ngrams=2)

In [4]:
Vectorizer.adapt(np.array(df['text']))

In [5]:
vocab = Vectorizer.get_vocabulary()

In [6]:
num_tokens = len(vocab)
print(num_tokens)

246970


In [7]:
#model = tf.keras.Sequential([
#    Vectorizer,
#    tf.keras.layers.Dense(32, activation='relu'),
#    tf.keras.layers.Dense(3, activation='softmax')
#])

In [8]:
def create_model(neurons=10, layers=2):
    model = tf.keras.Sequential()
    model.add(Vectorizer)
    
    for n in range(layers):
        model.add(tf.keras.layers.Dense(neurons, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.2))
    
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(1e-3),
             metrics=['accuracy'])
    return model

In [9]:
#model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
#             optimizer=tf.keras.optimizers.Adam(1e-3),
#             metrics=['accuracy'])

In [10]:
X = df['text']
y = df['author']

In [11]:
training_labels_bools = []

for author in y:
    if "EAP" == author:
        training_labels_bools.append([1,0,0])
    elif "HPL" == author:
        training_labels_bools.append([0,1,0])
    elif "MWS" == author:
        training_labels_bools.append([0,0,1])

In [12]:
y = training_labels_bools

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [14]:
X_train.values

array(['But this discovery was so great and overwhelming that all the steps by which I had been progressively led to it were obliterated, and I beheld only the result.',
       'I said to myself, "This is no dream, for by what means can I prove the greater reality of that other life in the house of stone and brick south of the sinister swamp and the cemetery on the low hillock, where the Pole Star peers into my north window each night?"',
       'A robin red breast dropt from the frosty branches of the trees, upon the congealed rivulet; its panting breast and half closed eyes shewed that it was dying: a hawk appeared in the air; sudden fear seized the little creature; it exerted its last strength, throwing itself on its back, raising its talons in impotent defence against its powerful enemy.',
       ...,
       'The manner in which Wyatt received this harmless pleasantry convinced me, at once, that he was mad.',
       'She first assured him of her boundless confidence; of this he mus

In [23]:
model = KerasClassifier(build_fn=create_model, batch_size=32, verbose=1)

batches = [16, 32, 64]
neurons = [10, 15, 20, 25]
layers = [1, 2, 4, 6]
epochs = [1]

param_grid = dict(neurons=neurons, epochs=epochs, layers=layers, batch_size=batches)
grid = GridSearchCV(estimator=model, scoring='neg_log_loss', param_grid=param_grid, n_jobs=1, cv=3, verbose=3)

grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] batch_size=16, epochs=1, layers=1, neurons=10 ...................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


612/612 [==============================] - 8s 13ms/step - loss: 0.6573 - accuracy: 0.7238
[CV]  batch_size=16, epochs=1, layers=1, neurons=10, score=-0.448, total=   9.7s
[CV] batch_size=16, epochs=1, layers=1, neurons=10 ...................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.7s remaining:    0.0s


612/612 [==============================] - 8s 13ms/step - loss: 0.6568 - accuracy: 0.7205
[CV]  batch_size=16, epochs=1, layers=1, neurons=10, score=-0.457, total=  10.0s
[CV] batch_size=16, epochs=1, layers=1, neurons=10 ...................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   19.7s remaining:    0.0s


612/612 [==============================] - 9s 14ms/step - loss: 0.6487 - accuracy: 0.7325
[CV]  batch_size=16, epochs=1, layers=1, neurons=10, score=-0.450, total=  10.5s
[CV] batch_size=16, epochs=1, layers=1, neurons=15 ...................
612/612 [==============================] - 11s 17ms/step - loss: 0.6260 - accuracy: 0.7431
[CV]  batch_size=16, epochs=1, layers=1, neurons=15, score=-0.432, total=  12.6s
[CV] batch_size=16, epochs=1, layers=1, neurons=15 ...................
612/612 [==============================] - 12s 19ms/step - loss: 0.6304 - accuracy: 0.7373
[CV]  batch_size=16, epochs=1, layers=1, neurons=15, score=-0.432, total=  13.6s
[CV] batch_size=16, epochs=1, layers=1, neurons=15 ...................
612/612 [==============================] - 12s 19ms/step - loss: 0.6174 - accuracy: 0.7444
[CV]  batch_size=16, epochs=1, layers=1, neurons=15, score=-0.448, total=  14.1s
[CV] batch_size=16, epochs=1, layers=1, neurons=20 ...................
612/612 [====================

612/612 [==============================] - 11s 17ms/step - loss: 0.9979 - accuracy: 0.4941
[CV]  batch_size=16, epochs=1, layers=6, neurons=10, score=-0.852, total=  12.8s
[CV] batch_size=16, epochs=1, layers=6, neurons=10 ...................
612/612 [==============================] - 11s 17ms/step - loss: 1.0030 - accuracy: 0.4891
[CV]  batch_size=16, epochs=1, layers=6, neurons=10, score=-0.872, total=  13.0s
[CV] batch_size=16, epochs=1, layers=6, neurons=10 ...................
612/612 [==============================] - 11s 16ms/step - loss: 0.9931 - accuracy: 0.5038
[CV]  batch_size=16, epochs=1, layers=6, neurons=10, score=-0.838, total=  12.7s
[CV] batch_size=16, epochs=1, layers=6, neurons=15 ...................
612/612 [==============================] - 14s 21ms/step - loss: 0.9138 - accuracy: 0.5942
[CV]  batch_size=16, epochs=1, layers=6, neurons=15, score=-0.609, total=  15.6s
[CV] batch_size=16, epochs=1, layers=6, neurons=15 ...................
612/612 [===================

KeyboardInterrupt: 

In [19]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('%f (%f) with: %r' % (mean, stdev, param))

-0.451206 (0.010626) with: {'epochs': 1, 'layers': 1, 'neurons': 10}
-0.444123 (0.001995) with: {'epochs': 1, 'layers': 1, 'neurons': 15}
-0.439087 (0.008581) with: {'epochs': 1, 'layers': 1, 'neurons': 20}
-0.429369 (0.005241) with: {'epochs': 1, 'layers': 1, 'neurons': 25}
-0.519594 (0.043255) with: {'epochs': 1, 'layers': 2, 'neurons': 10}
-0.469815 (0.001766) with: {'epochs': 1, 'layers': 2, 'neurons': 15}
-0.458230 (0.007228) with: {'epochs': 1, 'layers': 2, 'neurons': 20}
-0.449325 (0.004645) with: {'epochs': 1, 'layers': 2, 'neurons': 25}
-0.728966 (0.068606) with: {'epochs': 1, 'layers': 4, 'neurons': 10}
-0.637830 (0.018364) with: {'epochs': 1, 'layers': 4, 'neurons': 15}
-0.506345 (0.006745) with: {'epochs': 1, 'layers': 4, 'neurons': 20}
-0.499139 (0.009262) with: {'epochs': 1, 'layers': 4, 'neurons': 25}
-0.870501 (0.044818) with: {'epochs': 1, 'layers': 6, 'neurons': 10}
-0.807303 (0.037110) with: {'epochs': 1, 'layers': 6, 'neurons': 15}
-0.732095 (0.090398) with: {'epoch

In [20]:
scores = pd.DataFrame({
    "mean": means,
    "stdev": stds,
    "params": params
})
scores.sort_values('mean', ascending=False)

,mean,stdev,params
3,-0.429369,0.005241,"{'epochs': 1, 'layers': 1, 'neurons': 25}"
2,-0.439087,0.008581,"{'epochs': 1, 'layers': 1, 'neurons': 20}"
1,-0.444123,0.001995,"{'epochs': 1, 'layers': 1, 'neurons': 15}"
7,-0.449325,0.004645,"{'epochs': 1, 'layers': 2, 'neurons': 25}"
0,-0.451206,0.010626,"{'epochs': 1, 'layers': 1, 'neurons': 10}"
6,-0.458230,0.007228,"{'epochs': 1, 'layers': 2, 'neurons': 20}"
5,-0.469815,0.001766,"{'epochs': 1, 'layers': 2, 'neurons': 15}"
11,-0.499139,0.009262,"{'epochs': 1, 'layers': 4, 'neurons': 25}"
10,-0.506345,0.006745,"{'epochs': 1, 'layers': 4, 'neurons': 20}"
4,-0.519594,0.043255,"{'epochs': 1, 'layers': 2, 'neurons': 10}"


In [25]:
create_model(neurons=25, layers=1).fit(list(X_train), y_train, epochs=1, batch_size=64,
         validation_data=(list(X_test),y_test),
         validation_steps=10)

230/230 [==============================] - 9s 39ms/step - loss: 0.5626 - accuracy: 0.7675 - val_loss: 0.3762 - val_accuracy: 0.8547


In [26]:
X_sub = test_df['text']

NameError: name 'test_df' is not defined

In [24]:
sub_pred = model.predict(X_sub)
sub_pred

array([[5.6715729e-04, 1.9040395e-05, 9.9941385e-01],
       [1.0000000e+00, 2.1028919e-10, 4.9274198e-09],
       [2.4573201e-06, 9.9998271e-01, 1.4841148e-05],
       ...,
       [9.9999821e-01, 6.4516219e-09, 1.8274793e-06],
       [8.3473593e-02, 1.1622645e-07, 9.1652632e-01],
       [3.9207583e-04, 9.9960798e-01, 4.3928186e-09]], dtype=float32)

In [25]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "EAP": sub_pred[:, 0],
    "HPL": sub_pred[:, 1],
    "MWS": sub_pred[:, 2]
})
submission.head()

,id,EAP,HPL,MWS
0,id02310,0.000567,1.904040e-05,9.994138e-01
1,id24541,1.000000,2.102892e-10,4.927420e-09
2,id00134,0.000002,9.999827e-01,1.484115e-05
3,id27757,0.999981,1.609015e-05,3.156577e-06
4,id04081,0.176412,2.596113e-01,5.639762e-01


In [26]:
#submission.to_csv("submission.csv", index=False)